# Transformer 实验与调参手册 (AG NEWS)

## 目录
- [环境准备](#环境准备)
- [单次 Baseline 训练](#单次-Baseline-训练)
- [超参数网格搜索](#超参数网格搜索)
- [结果汇总表](#结果汇总表)
- [曲线可视化](#曲线可视化)
- [拆零件小实验](#拆零件小实验)
- [Attention 可视化](#Attention-可视化)
- [结论总结](#结论总结)


## 环境准备


In [ ]:
import os, sys, torch, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import numpy as np
import subprocess
import json
from pathlib import Path

sys.path.append("..")      # 项目根
%load_ext autoreload
%autoreload 2

print(torch.__version__, torch.cuda.is_available())


## 单次 Baseline 训练

首先运行一次基础训练，确保代码无错误且能成功训练。


In [ ]:
# 运行基础训练
!python ../train/train_transformer.py --epochs 5 --batch-size 64 --lr 1e-3 --embed-dim 128


## 超参数网格搜索

定义一组系统化的超参数组合，进行网格搜索实验。


In [ ]:
# 定义超参数网格
grid = [
    {"embed_dim": d, "num_heads": h, "lr": lr}
    for d, h in [(64, 2), (128, 4), (256, 8)]
    for lr in [5e-4, 1e-3]
]
results = []


In [ ]:
# 函数用于解析输出中的验证准确率
def parse_output(output_str):
    lines = output_str.split('\n')
    val_acc = None
    for line in lines:
        if "Val loss" in line:
            parts = line.split(',')
            if len(parts) > 1:
                acc_part = parts[1].strip()
                val_acc = float(acc_part.replace('Acc: ', '').replace('%', ''))
    return val_acc


In [ ]:
# 循环执行每个超参数组合的实验
for i, params in enumerate(grid):
    print(f"\n实验 {i+1}/{len(grid)}: {params}")
    
    # 构建命令
    cmd = [
        "python", "../train/train_transformer.py",
        "--epochs", "3",
        "--batch-size", "64",
        "--embed-dim", str(params['embed_dim']),
        "--num-heads", str(params['num_heads']),
        "--lr", str(params['lr'])
    ]
    
    # 执行命令并捕获输出
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate()
    
    # 解析验证准确率
    val_acc = parse_output(stdout)
    
    # 将结果添加到列表
    result = params.copy()
    result['val_acc'] = val_acc
    results.append(result)
    
    print(f"验证准确率: {val_acc:.2f}%")


## 结果汇总表

将网格搜索结果整理成 DataFrame 并排序。


In [ ]:
# 转换为 DataFrame
df = pd.DataFrame(results)

# 按验证准确率降序排序
df_sorted = df.sort_values(by='val_acc', ascending=False)
display(df_sorted)


## 曲线可视化

绘制超参数对性能影响的曲线图。


In [ ]:
plt.figure(figsize=(12, 8))

# 1. Embedding维度对准确率影响
sns.lineplot(data=df, x="embed_dim", y="val_acc", hue="num_heads", style="lr")
plt.title("Embedding 维度 / Head 数 / LR 对准确率影响")
plt.xlabel("Embedding 维度")
plt.ylabel("验证准确率 (%)")
plt.grid(True, linestyle='--', alpha=0.7)
plt.savefig('../outputs/transformer/param_impact.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# 2. 学习率对不同模型规模的影响
g = sns.catplot(
    data=df, x="lr", y="val_acc", 
    col="embed_dim", hue="num_heads",
    kind="bar", height=5, aspect=0.8
)
g.set_axis_labels("学习率", "验证准确率 (%)")
g.set_titles("Embedding维度: {col_name}")
plt.savefig('../outputs/transformer/lr_impact.png', dpi=300, bbox_inches='tight')
plt.show()


## 拆零件小实验

进行6种结构变体实验，观察各组件对模型性能的影响。


In [ ]:
# 定义消融实验列表
ablations = [
    "no_pe",        # 关闭位置编码
    "single_head",  # 只用单头注意力
    "no_ffn",       # 移除前馈网络
    "freeze_emb",   # 冻结embedding
    "no_dropout",   # 关闭dropout
    "clip_grad"     # 梯度裁剪
]

# 获取最佳配置作为基准
best_config = df_sorted.iloc[0].to_dict()
embed_dim = int(best_config['embed_dim'])
num_heads = int(best_config['num_heads'])
lr = best_config['lr']

ablation_results = []


In [ ]:
# 先运行一次基础模型（无消融）作为对照
print("运行基准模型（无消融）...")
cmd = [
    "python", "../train/train_transformer.py",
    "--epochs", "3",
    "--batch-size", "64",
    "--embed-dim", str(embed_dim),
    "--num-heads", str(num_heads),
    "--lr", str(lr)
]
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
stdout, stderr = process.communicate()
val_acc = parse_output(stdout)
ablation_results.append({
    "ablation": "baseline",
    "description": "基准模型（无消融）",
    "val_acc": val_acc
})
print(f"基准模型验证准确率: {val_acc:.2f}%")


In [ ]:
# 运行各种消融实验
descriptions = {
    "no_pe": "关闭位置编码",
    "single_head": "只用单头注意力",
    "no_ffn": "移除前馈网络",
    "freeze_emb": "冻结embedding",
    "no_dropout": "关闭dropout",
    "clip_grad": "梯度裁剪"
}

for ablation in ablations:
    print(f"\n运行消融实验: {ablation} - {descriptions[ablation]}")
    cmd = [
        "python", "../train/train_transformer.py",
        "--epochs", "3",
        "--batch-size", "64",
        "--embed-dim", str(embed_dim),
        "--num-heads", str(num_heads),
        "--lr", str(lr),
        "--ablation", ablation
    ]
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate()
    val_acc = parse_output(stdout)
    ablation_results.append({
        "ablation": ablation,
        "description": descriptions[ablation],
        "val_acc": val_acc
    })
    print(f"验证准确率: {val_acc:.2f}%")


In [ ]:
# 将消融实验结果整理为DataFrame
ablation_df = pd.DataFrame(ablation_results)

# 计算相对于基准的性能变化百分比
# 获取基准值
baseline_rows = ablation_df[ablation_df['ablation'] == 'baseline']
if len(baseline_rows) > 0:
    baseline_acc = list(baseline_rows['val_acc'])[0]
else:
    baseline_acc = 0
ablation_df['rel_change'] = (ablation_df['val_acc'] - baseline_acc) / baseline_acc * 100

# 排序并显示结果
ablation_sorted = ablation_df.sort_values(by='val_acc', ascending=False)
display(ablation_sorted)


In [ ]:
# 可视化消融实验结果
plt.figure(figsize=(12, 6))

# 不包含baseline的数据
plot_df = ablation_df[ablation_df['ablation'] != 'baseline'].copy()

# 根据相对变化排序 (这将正常工作，尽管IDE可能显示警告)
# sort_values是pandas的标准方法，运行时不会出错
plot_df = plot_df.sort_values('rel_change')

# 创建柱状图
bars = plt.bar(plot_df['description'], plot_df['rel_change'])

# 为负值设置不同颜色
for i, v in enumerate(plot_df['rel_change']):
    if v < 0:
        bars[i].set_alpha(0.7)

plt.axhline(y=0, color='r', linestyle='-', alpha=0.3)
plt.title(f"各种结构变体对模型性能的影响 (相对于基准 {baseline_acc:.2f}%)", fontsize=14)
plt.xlabel("结构变体")
plt.ylabel("相对准确率变化 (%)")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.savefig('../outputs/transformer/ablation_results.png', dpi=300, bbox_inches='tight')
plt.show()


## Attention 可视化

加载最高分模型，选择一条验证集样本，可视化第1层第1个头的注意力热力图。


In [ ]:
import sys
sys.path.append('..')
import torch
from models.transformer import Transformer
from utils.text_dataloader import get_ag_news_dataloader
from pathlib import Path


In [ ]:
# 加载最佳模型
# 获取验证数据加载器
data_dir = Path("../data")
val_loader, vocab, vocab_size, num_classes = get_ag_news_dataloader(
    batch_size=1,  # 单条样本
    max_len=128,   # 限制长度以便可视化
    train=False,
    root=str(data_dir)
)

# 根据最佳超参数创建模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Transformer(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    num_heads=num_heads,
    hidden_dim=embed_dim*4,  # 标准Transformer通常使用4倍embed_dim作为FFN隐藏层
    num_layers=3,
    num_classes=num_classes
)

# 加载最佳检查点
best_model_path = Path("../outputs/transformer/transformer_best.pth")
if best_model_path.exists():
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    print(f"已加载最佳模型: {best_model_path}")
else:
    print(f"找不到最佳模型文件，使用未训练模型")

model = model.to(device)


In [ ]:
# 提取单个Transformer层注意力权重的钩子函数
class AttentionHook:
    def __init__(self):
        self.attn_weights = None
        
    def __call__(self, module, module_input, module_output):
        # PyTorch TransformerEncoder层的attention weights在运行时不保存
        # 我们需要修改模型结构以访问它们
        # 这里仅为示例，实际使用时需调整
        self.attn_weights = None  # 稍后会在前向传播中手动提取


# 修改Transformer模型来返回注意力权重
def forward_with_attention(model, x, src_key_padding_mask=None):
    """修改的前向传播，返回注意力权重"""
    # 词嵌入 [batch_size, seq_len] -> [batch_size, seq_len, embed_dim]
    embedded = model.embedding(x)
    
    # 添加位置编码
    embedded = model.pos_encoder(embedded)
    
    # 我们需要访问TransformerEncoder的第一层的第一个注意力头
    # 由于PyTorch没有直接暴露这个API，我们需要手动添加钩子或修改模型
    # 这里是一个近似方法，实际实现可能需要更深入地修改模型
    
    # 使用第一个编码器层的self-attention模块
    first_layer = model.transformer_encoder.layers[0]
    
    # 1. 应用自注意力，并保存注意力权重
    # 这部分逻辑通常在TransformerEncoderLayer的内部
    q = first_layer.self_attn.q_proj(embedded)
    k = first_layer.self_attn.k_proj(embedded)
    v = first_layer.self_attn.v_proj(embedded)
    
    # 重塑为多头形式
    batch_size, seq_len = embedded.shape[0], embedded.shape[1]
    head_dim = model.transformer_encoder.layers[0].self_attn.head_dim
    num_heads = model.transformer_encoder.layers[0].self_attn.num_heads
    
    q = q.view(batch_size, seq_len, num_heads, head_dim).transpose(1, 2)
    k = k.view(batch_size, seq_len, num_heads, head_dim).transpose(1, 2)
    v = v.view(batch_size, seq_len, num_heads, head_dim).transpose(1, 2)
    
    # 计算注意力分数
    scores = torch.matmul(q, k.transpose(-2, -1)) / (head_dim ** 0.5)
    
    # 应用掩码（如果有）
    if src_key_padding_mask is not None:
        # 扩展掩码以匹配分数的形状
        mask = src_key_padding_mask.unsqueeze(1).unsqueeze(2)
        scores = scores.masked_fill(mask, float("-inf"))
    
    # 应用softmax得到注意力权重
    attn_weights = torch.nn.functional.softmax(scores, dim=-1)
    
    # 提取第一个头的注意力权重
    first_head_attn = attn_weights[0, 0].cpu().detach()
    
    # 继续常规前向传播
    # 由于我们不需要完整的前向传播结果，这里简化了步骤
    
    return first_head_attn


In [ ]:
# 选择一条验证样本
model.eval()
with torch.no_grad():
    # 获取一个批次
    for tokens, labels, masks in val_loader:
        tokens, labels, masks = tokens.to(device), labels.to(device), masks.to(device)
        
        # 使用修改后的前向传播函数获取注意力权重
        attention_weights = forward_with_attention(model, tokens, src_key_padding_mask=masks)
        
        # 获取真实标记（而不是填充标记）
        valid_seq_len = (~masks[0]).sum().item()
        
        # 可视化注意力热力图
        plt.figure(figsize=(10, 8))
        
        # 只显示有效的标记（不显示填充标记）
        valid_attn = attention_weights[:valid_seq_len, :valid_seq_len]
        
        # 创建热力图
        sns.heatmap(valid_attn, cmap="viridis")
        plt.title(f"第1层第1个头的Self-Attention热力图 (类别: {labels.item()+1})")
        plt.xlabel("Token位置")
        plt.ylabel("Token位置")
        plt.tight_layout()
        plt.savefig('../outputs/transformer/attention_heatmap.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # 一条样本就够了
        break


## 结论总结

1. **超参数实验结论**
   - 最佳超参数组合及其性能表现
   - embedding维度与head数量的关系
   - 学习率对不同模型规模的影响

2. **结构变体实验结论**
   - 各组件对模型性能的影响排序
   - 哪些组件是必不可少的，哪些影响较小

3. **注意力可视化分析**
   - 模型关注了哪些位置的token
   - 是否存在明显的注意力模式

4. **TODO 与改进方向**
   - 尝试更多超参数组合
   - 增加序列长度或批量大小
   - 尝试不同的位置编码方案
   - 探索预训练与微调策略
